# Analysis

In [16]:
import json

with open('diffs.json', 'r') as f:
    diffs = json.load(f)

with open('codes.json', 'r') as cf:
    codes = json.load(cf)

## Dataset

In [17]:
TOTAL = 538
retrieved = len(diffs)
with_saving = len([d for d in diffs if 'saving' in d['existing_codes']])
print('retrieved commits:', f'{retrieved}/{TOTAL}', f'({retrieved/TOTAL*100:.2f}%)')
print('originally coded with `saving`:', f'{with_saving}/{retrieved}', f'({with_saving/retrieved*100:.2f}%)')

n_processed = sum(len(d['codes']) != 0 for d in diffs)
n_processed_with_saving = sum(len(d['codes']) != 0 and 'saving' in d['existing_codes'] for d in diffs)
print('processed commits:', f'{n_processed}/{retrieved}', f'({n_processed/retrieved*100:.2f}%)')
print('processed commits originally coded with `saving`:', f'{n_processed_with_saving}/{with_saving}', f'({n_processed_with_saving/with_saving*100:.2f}%)')

not_relevant = sum('not_relevant' in d['codes'] for d in diffs)
print('commits coded with `not_relevant`:', f'{not_relevant}/{n_processed}', f'({not_relevant/retrieved*100:.2f}%)')

avg_files = sum(len(d['files']) for d in diffs) / len(diffs)
print('files affected (avg):', f'{avg_files:.2f}')

retrieved commits: 499/538 (92.75%)
originally coded with `saving`: 342/499 (68.54%)
processed commits: 490/499 (98.20%)
processed commits originally coded with `saving`: 336/342 (98.25%)
commits coded with `not_relevant`: 177/490 (35.47%)
files affected (avg): 4.42


## Codes

In [18]:
import polars as pl

codes_df = pl.DataFrame(codes) \
            .with_columns(count=sum(pl.col('name').is_in(d['codes']) for d in diffs)) \
            .with_columns(percentage=pl.col('count') / n_processed * 100) \
            .sort(by=pl.col('count'), descending=True)

print(codes_df)

codes_df.drop("percentage").write_csv("codes.csv")

shape: (122, 4)
┌─────────────────────────────┬───────────────────────────────────┬───────┬────────────┐
│ name                        ┆ description                       ┆ count ┆ percentage │
│ ---                         ┆ ---                               ┆ ---   ┆ ---        │
│ str                         ┆ str                               ┆ i32   ┆ f64        │
╞═════════════════════════════╪═══════════════════════════════════╪═══════╪════════════╡
│ aws                         ┆ Specific to Amazon Web Services   ┆ 356   ┆ 72.653061  │
│ not_relevant                ┆ The commit contents do not refle… ┆ 177   ┆ 36.122449  │
│ cheaper_instance            ┆ Use a cheaper compute instance    ┆ 73    ┆ 14.897959  │
│ gcp                         ┆ Specific to Google Cloud Platfor… ┆ 63    ┆ 12.857143  │
│ azure                       ┆ Specific to Azure                 ┆ 51    ┆ 10.408163  │
│ …                           ┆ …                                 ┆ …     ┆ …          │
│ rem

### Cloud-specific Codes

In [19]:
clouds = {'digitalocean', 'aws', 'gcp', 'azure', 'alicloud', 'scaleway', 'hcloud', 'ibm', 'oracle', 'ovh'}
print(codes_df.filter(pl.col('name').is_in(clouds)).drop('description'))

shape: (10, 3)
┌──────────────┬───────┬────────────┐
│ name         ┆ count ┆ percentage │
│ ---          ┆ ---   ┆ ---        │
│ str          ┆ i32   ┆ f64        │
╞══════════════╪═══════╪════════════╡
│ aws          ┆ 356   ┆ 72.653061  │
│ gcp          ┆ 63    ┆ 12.857143  │
│ azure        ┆ 51    ┆ 10.408163  │
│ digitalocean ┆ 12    ┆ 2.44898    │
│ scaleway     ┆ 3     ┆ 0.612245   │
│ hcloud       ┆ 2     ┆ 0.408163   │
│ ibm          ┆ 2     ┆ 0.408163   │
│ oracle       ┆ 2     ┆ 0.408163   │
│ alicloud     ┆ 1     ┆ 0.204082   │
│ ovh          ┆ 1     ┆ 0.204082   │
└──────────────┴───────┴────────────┘


### Uniqueness

Determine how many unique owners the code occurs for, to see if there is actually a pattern across developers/organizations.

In [29]:
def repo_owner(diff: dict[str, str]) -> str:
    _, _, _, owner, *_ = diff['url'].split('/')
    return owner

n_owners = len(set(repo_owner(d) for d in diffs))
print('number of unique owners:', n_owners)

owners = [
    {
        'name': c['name'],
        'owners': list(set(repo_owner(d) for d in diffs if c['name'] in d['codes']))
    }
    for c in codes
]
owners_df = pl.DataFrame(owners) \
                .with_columns(n_owners=pl.col('owners').list.len()) \
                .sort(by=pl.col('n_owners'), descending=True)

filtered = owners_df.filter(pl.col('name').is_in(clouds).not_()).filter(pl.col('n_owners') >= 2)

print(filtered)

#filtered.drop('owners').write_csv('unique_owners.csv')

number of unique owners: 352
shape: (41, 3)
┌────────────────────────────┬───────────────────────────────────┬──────────┐
│ name                       ┆ owners                            ┆ n_owners │
│ ---                        ┆ ---                               ┆ ---      │
│ str                        ┆ list[str]                         ┆ u32      │
╞════════════════════════════╪═══════════════════════════════════╪══════════╡
│ not_relevant               ┆ ["stuartsan", "picatz", … "jsnid… ┆ 122      │
│ cheaper_instance           ┆ ["yardbirdsax", "deadlysyn", … "… ┆ 61       │
│ new_gen                    ┆ ["yardbirdsax", "dwp", … "kmishr… ┆ 25       │
│ dynamo_on_demand           ┆ ["MichaelDeCorte", "jenkins-x", … ┆ 19       │
│ remove_nat_gateway         ┆ ["CheesecakeLabs", "joshuaspence… ┆ 18       │
│ …                          ┆ …                                 ┆ …        │
│ reduce_node_count          ┆ ["giantswarm", "armand1m"]        ┆ 2        │
│ remove_status_chec

### Undefined codes

List codes which are attached to commits but not defined in `codes.json`.

In [21]:
listed_codes = set(c['name'] for c in codes)
for d in diffs:
    for c in d['codes']:
        if c not in listed_codes:
            print(f'{c} is not listed')